In [1]:
!pip install beautifulsoup4 -q

In [24]:
!pip install lxml

In [31]:
!pip install selenium -q

In [1]:
import requests
import pandas as pd
import csv
from bs4 import BeautifulSoup
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [61]:
url = 'https://letterboxd.com/films/popular/page/1/'
responce = requests.get(url)

In [62]:
responce

<Response [200]>

In [63]:
soup = BeautifulSoup(responce.text, 'html.parser')

In [64]:
movie_containers = soup.find_all('div', class_='film-poster')

In [65]:
movie_containers

[]

In [113]:
!pip install webdriver-manager -q

In [8]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd

# Setup Selenium WebDriver (assuming you have the ChromeDriver installed)
chrome_driver_path = '/home/seppoku/ds_bootcamp/ds-phase-2/find_my_movie/chromedriver/chromedriver'  # Change this path to your chromedriver path
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

# Function to extract movie details from a single page
def parse_movie_page(soup):
    movies = []

    for movie_item in soup.find_all('div', class_='film-poster'):
        title_tag = movie_item.find('img')
        title = title_tag['alt'].strip() if title_tag else 'N/A'

        # Movie link
        movie_link = 'https://letterboxd.com' + movie_item.find('a')['href']

        # Visit movie page to get additional details
        driver.get(movie_link)
        movie_soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Extract the year of release from the main page
        year_tag = movie_item.find('span', class_='year')
        year = year_tag.text.strip() if year_tag else 'N/A'

        # Extract the rating
        rating_tag = movie_soup.find('meta', itemprop='ratingValue')
        rating = rating_tag['content'].strip() if rating_tag else 'N/A'

        # Extract the description
        description_tag = movie_soup.find('div', class_='truncate')
        description = description_tag.text.strip() if description_tag else 'N/A'

        # Extract the film poster URL
        poster_tag = movie_soup.find('meta', property='og:image')
        poster_url = poster_tag['content'].strip() if poster_tag else 'N/A'

        movies.append({
            'Title': title,
            'Year': year,
            'Rating': rating,
            'Description': description,
            'Poster URL': poster_url,
            'Page URL': movie_link
        })

        time.sleep(1)  # Be respectful to the server; don't overload with requests

    return movies

# Function to save data to CSV
def save_to_csv(data, filename='movies.csv'):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, mode='a', header=not pd.io.common.file_exists(filename))  # Append to CSV

# Main function to scrape the movies
def scrape_letterboxd_movies(max_movies=5000):
    url = 'https://letterboxd.com/films/popular/size/small/page/35/'
    driver.get(url)
    all_movies = []

    page = 35
    total_parsed = 0  # Initialize total parsed movies count

    while total_parsed < max_movies:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        movies = parse_movie_page(soup)
        all_movies.extend(movies)

        # Update total parsed movies count
        total_parsed += len(movies)
        print(f'Parsed {len(movies)} movies from page {page}. Total parsed: {total_parsed}')

        # Save to CSV after every 20 movies
        while len(all_movies) >= 20:
            save_to_csv(all_movies[:20])  # Save the first 20 movies
            print(f'Saved 20 movies to CSV. Total saved: {total_parsed}')
            all_movies = all_movies[20:]  # Remove the saved movies from the list

        if total_parsed >= max_movies:
            break

        # Navigate to the next page
        page += 1
        next_page_url = f'https://letterboxd.com/films/popular/size/small/page/{page}/'
        driver.get(next_page_url)
        time.sleep(3)  # Wait for the page to load

    # Save any remaining movies that were not saved
    if all_movies:
        save_to_csv(all_movies)  # Save any remaining movies
        print(f'Saved remaining {len(all_movies)} movies to CSV. Total saved: {total_parsed + len(all_movies)}')

    print(f'Total movies scraped: {total_parsed}.')
    driver.quit()

# Run the scraper
scrape_letterboxd_movies()




WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=127.0.6533.119)
Stacktrace:
#0 0x63abe4a696ca <unknown>
#1 0x63abe473a600 <unknown>
#2 0x63abe4720fe8 <unknown>
#3 0x63abe4720ec3 <unknown>
#4 0x63abe473c8f2 <unknown>
#5 0x63abe47ca8f3 <unknown>
#6 0x63abe47ac643 <unknown>
#7 0x63abe477cd31 <unknown>
#8 0x63abe477d79e <unknown>
#9 0x63abe4a3125b <unknown>
#10 0x63abe4a351f2 <unknown>
#11 0x63abe4a1e615 <unknown>
#12 0x63abe4a35d82 <unknown>
#13 0x63abe4a0325f <unknown>
#14 0x63abe4a58e68 <unknown>
#15 0x63abe4a59040 <unknown>
#16 0x63abe4a6849c <unknown>
#17 0x731e40c9ca94 <unknown>
#18 0x731e40d29c3c <unknown>


In [74]:
df = pd.read_csv('movies.csv')

In [82]:
df

,Title,Year,Rating,Description,Poster URL,Page URL
0,Barbie,NaN,NaN,Barbie and Ken are having the time of their li...,https://a.ltrbxd.com/resized/sm/upload/mm/bt/i...,https://letterboxd.com/film/barbie/
1,Parasite,NaN,NaN,"All unemployed, Ki-taek’s family takes peculia...",https://a.ltrbxd.com/resized/sm/upload/oi/ha/7...,https://letterboxd.com/film/parasite-2019/
2,Everything Everywhere All at Once,NaN,NaN,An aging Chinese immigrant is swept up in an i...,https://a.ltrbxd.com/resized/sm/upload/qo/9b/x...,https://letterboxd.com/film/everything-everywh...
3,Fight Club,NaN,NaN,A ticking-time-bomb insomniac and a slippery s...,https://a.ltrbxd.com/resized/sm/upload/b0/iz/e...,https://letterboxd.com/film/fight-club/
4,La La Land,NaN,NaN,"Mia, an aspiring actress, serves lattes to mov...",https://a.ltrbxd.com/resized/sm/upload/a6/th/c...,https://letterboxd.com/film/la-la-land/
5,Interstellar,NaN,NaN,The adventures of a group of explorers who mak...,https://a.ltrbxd.com/resized/sm/upload/r4/0u/o...,https://letterboxd.com/film/interstellar/
6,Oppenheimer,NaN,NaN,The story of J. Robert Oppenheimer’s role in t...,https://a.ltrbxd.com/resized/sm/upload/mn/uu/o...,https://letterboxd.com/film/oppenheimer-2023/
7,Joker,NaN,NaN,"During the 1980s, a failed stand-up comedian i...",https://a.ltrbxd.com/resized/sm/upload/fe/g8/y...,https://letterboxd.com/film/joker-2019/
8,Dune,NaN,NaN,"Paul Atreides, a brilliant and gifted young ma...",https://a.ltrbxd.com/resized/sm/upload/a6/8b/5...,https://letterboxd.com/film/dune-2021/
9,Pulp Fiction,NaN,NaN,"A burger-loving hit man, his philosophical par...",https://a.ltrbxd.com/resized/sm/upload/z6/e0/v...,https://letterboxd.com/film/pulp-fiction/
